In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def generate_data(num_samples=1000):
    np.random.seed(42)
    x = np.random.uniform(-2, 2, num_samples)
    y = np.random.uniform(-2, 2, num_samples)
    z = np.random.uniform(-2, 2, num_samples)
    f_xyz = np.sin(x) + np.cos(y) * np.power(z, 2)
    features = np.vstack((x, y, z)).T
    outputs = f_xyz
    return features, outputs


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

class ThreeLayerNN(tf.Module):
    def __init__(self, input_size=3, hidden_size=64, output_size=1):
        super().__init__()
        self.W1 = tf.Variable(tf.random.normal([input_size, hidden_size], stddev=0.01))
        self.b1 = tf.Variable(tf.zeros([hidden_size]))
        self.W2 = tf.Variable(tf.random.normal([hidden_size, hidden_size], stddev=0.01))
        self.b2 = tf.Variable(tf.zeros([hidden_size]))
        self.W3 = tf.Variable(tf.random.normal([hidden_size, output_size], stddev=0.01))
        self.b3 = tf.Variable(tf.zeros([output_size]))

    def __call__(self, X):
        Z1 = tf.add(tf.matmul(X, self.W1), self.b1)
        A1 = tf.nn.relu(Z1)
        Z2 = tf.add(tf.matmul(A1, self.W2), self.b2)
        A2 = tf.nn.relu(Z2)
        Z3 = tf.add(tf.matmul(A2, self.W3), self.b3)
        return Z3

def compute_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

class ModelTrainer:
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer

    @tf.function
    def train_step(self, X, y):
        with tf.GradientTape() as tape:
            y_pred = self.model(X)
            loss = compute_loss(y, y_pred)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return loss

    def train(self, train_dataset, num_epochs=100):
        for epoch in range(num_epochs):
            epoch_loss_avg = tf.keras.metrics.Mean()
            for X_batch, y_batch in train_dataset:
                loss = self.train_step(X_batch, y_batch)
                epoch_loss_avg.update_state(loss)
            print(f"Epoch {epoch + 1}: Loss = {epoch_loss_avg.result().numpy()}")

# Generate and prepare data
features, outputs = generate_data()  # Assume this function is defined elsewhere
X_train, X_test, y_train, y_test = train_test_split(features, outputs, test_size=0.2, random_state=42)
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.float32)

# Setup training
model = ThreeLayerNN()
optimizer = tf.optimizers.Adam(learning_rate=0.001)
trainer = ModelTrainer(model, optimizer)

# Convert training data to TensorFlow Dataset for easy batching
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tf, y_train_tf))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

# Train the model
trainer.train(train_dataset, num_epochs=100)

# Evaluate the model
y_pred = model(X_test_tf)
test_loss = compute_loss(y_test_tf, y_pred)
print(f"Test Loss: {test_loss.numpy()}")


Epoch 1: Loss = 1.9990772008895874
Epoch 2: Loss = 1.7712515592575073
Epoch 3: Loss = 1.683562159538269
Epoch 4: Loss = 1.6808720827102661
Epoch 5: Loss = 1.6830840110778809
Epoch 6: Loss = 1.682672381401062
Epoch 7: Loss = 1.680635690689087
Epoch 8: Loss = 1.6825085878372192
Epoch 9: Loss = 1.6802246570587158
Epoch 10: Loss = 1.6806625127792358
Epoch 11: Loss = 1.6823805570602417
Epoch 12: Loss = 1.6777071952819824
Epoch 13: Loss = 1.6781960725784302
Epoch 14: Loss = 1.6761876344680786
Epoch 15: Loss = 1.680379033088684
Epoch 16: Loss = 1.6795252561569214
Epoch 17: Loss = 1.6812105178833008
Epoch 18: Loss = 1.6782373189926147
Epoch 19: Loss = 1.681700587272644
Epoch 20: Loss = 1.6768536567687988
Epoch 21: Loss = 1.6787195205688477
Epoch 22: Loss = 1.6810320615768433
Epoch 23: Loss = 1.6808714866638184
Epoch 24: Loss = 1.6798713207244873
Epoch 25: Loss = 1.6777795553207397
Epoch 26: Loss = 1.6794718503952026
Epoch 27: Loss = 1.6774235963821411
Epoch 28: Loss = 1.6833000183105469
Epoch 